# Importing required packages
* tesnsorflow, numpy, pandas
* cv2 for drawing rectangle on images
* tdqm for showing progress bar
* shutil for copying images

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os
import random

from tqdm.auto import tqdm
import shutil as sh
from PIL import Image

# Importing images for training our custom traning set
* **train_dir**: training images 
* **test_dir**: testing images
* **data**: value of boxes co-ordinates on images(xmin,ymin,xmax,ymax)

In [ ]:
train_dir = "/kaggle/input/car-object-detection/data/training_images/"
test_dir = "/kaggle/input/car-object-detection/data/testing_images/"

print('Image(train): ', len(os.listdir(train_dir)))
print('Image(test): ', len(os.listdir(test_dir)))

In [ ]:
data = pd.read_csv("/kaggle/input/car-object-detection/data/train_solution_bounding_boxes (1).csv")

In [ ]:
data.head()

In [ ]:
print(data.shape)
data.nunique()

# Showing training Images with green boxes on car

In [ ]:
IMAGE_SHAPE = 0
for i in range(4):
    rand = np.random.randint(0, data.shape[0])
    random_row = data.iloc[rand]
    image = plt.imread(train_dir + random_row[0])
    plt.figure()
    plt.imshow(image)
    IMAGE_SHAPE = image.shape
    print(image.shape)
    print('Name,xmin,ymin,xmax,ymax:',random_row)
    point1 = (int(random_row[1]), int(random_row[2]))
    point2 = (int(random_row[3]), int(random_row[4]))
    cv2.rectangle(image, point1, point2, color=(0, 255, 0), thickness=3)
    plt.figure()
    plt.imshow(image)

# Cloning yolov5 from github

**Link:** https://github.com/ultralytics/yolov5 

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!cd yolov5
!pip install -r ./yolov5/requirements.txt

# Data processing

Converting (xmin,ymin,xmax,ymax) to (x_center,y_center, w, h) as yolo5 required that format.

In [ ]:
im_h, im_w, num_chanels = IMAGE_SHAPE
data["x_center"] = (data["xmax"] + data["xmin"])/2
data["y_center"] = (data["ymax"] + data["ymin"])/2
data["w"] = data["xmax"] - data["xmin"]
data["h"] = data["ymax"] - data["ymin"]
data["classes"] = 0
#Normalizing the values with image height and width
data["x_center"] = data["x_center"]/im_w
data["w"] = data["w"]/im_w
data["y_center"] = data["y_center"]/im_h
data["h"] = data["h"]/im_h
data.head()

In [ ]:
index = list(data.image)
print(len(index))

# Training with yolov5 model

First we are going create a folder "data/fold0" and divide our training images into train and validation set. We are going to create a txt file for each images with box co-ordinates(x_center, y_center, w, h).

**STEP-1: Creating a txt file for each images inside labels folder**

validation set: "data/fold0/labels/val/"

training set: "data/fold0/labels/train/"

**STEP-2: Copying images from each Input to folder**

validation images: "data/fold0/images/val/"

training images: "data/fold0/images/train/"

**STEP-3: Creating dataset.yaml**

Creating a dataset.yaml file as required for custom training the yolov5 model,which contains path to training data and validation data.

**STEP-4: Training the model with custom data**

We are going to train the model with:

    - Batch size = 12
    - Epochs = 12
    - CONF_TRESHOLD = 0.6
    - IOU_THRESHOLD = 0.5
    - MODEL = 'yolov5x.pt' (large model for better output)

In [ ]:
if not os.path.exists("data"):
    os.makedirs("data")
    os.makedirs("data/fold{}".format(0))

In [ ]:
source = 'training_images'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(data.groupby('image')):
            if name in val_index:
                path2save = 'val/'
            else:
                path2save = 'train/'
            if not os.path.exists('data/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('data/fold{}/labels/'.format(fold)+path2save)
            with open('data/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('data/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('data/fold{}/images/{}'.format(fold,path2save))
            sh.copy("/kaggle/input/car-object-detection/data/{}/{}".format(source,name),'data/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))
        print("Copy done")
        print('Image(train): ', len(os.listdir('data/fold0/images/train/')))
        print('Image(test): ', len(os.listdir('data/fold0/images/val/')))

In [ ]:
%%writefile dataset.yaml
path: /kaggle/working/data/fold0
train: /kaggle/working/data/fold0/images/train/
val: /kaggle/working/data/fold0/images/val/
test:

names:
    0: car

In [ ]:
IMG_SIZE = IMAGE_SHAPE[1]
CONF_TRESHOLD = 0.6
IOU_THRESHOLD = 0.5
MODEL = 'yolov5x.pt'

In [ ]:
import time

start = time.perf_counter()

!python yolov5/train.py --batch 12 --epochs 12 --data dataset.yaml --weights {MODEL} --name my_model_5x

end = time.perf_counter()

total_time = end - start

print(f'Training time for {MODEL} model is: {total_time:0.2f} seconds')

In [ ]:
!ls yolov5/runs/train/my_model_5x/

In [ ]:
Image.open("yolov5/runs/train/my_model_5x/F1_curve.png")

In [ ]:
Image.open("yolov5/runs/train/my_model_5x/confusion_matrix.png")

# Detecting cars on test images

We are now going to use our custom trained model to detect cars on our testing images. The wieghts of our train model are inside "yolov5/runs/train/my_model_5x/weights/best.pt".

In [ ]:
!python ./yolov5/detect.py --img {IMG_SIZE} --conf {CONF_TRESHOLD} --iou-thres {IOU_THRESHOLD} --source {test_dir} --weights yolov5/runs/train/my_model_5x/weights/best.pt

In [ ]:
!ls yolov5/runs/detect/exp/

In [ ]:
pred_dir = './yolov5/runs/detect/exp'
file_paths = []
flist = os.listdir(pred_dir)
for file in flist:
    fpath = os.path.join(pred_dir, file)
    file_paths.append(fpath)

plt.figure(figsize=(30, 20))
for i in range(16):
    random = np.random.randint(0, len(file_paths) - 1)
    plt.subplot(4, 4, i + 1)
    img_path = file_paths[random]
    img = plt.imread(img_path)
    plt.imshow(img)
    #plt.title('Predictions', size=10, color="black") 
    plt.xticks([])
    plt.yticks([])
    
plt.show()